In [1]:
import numpy as np
import pandas as pd
from loguru import logger
from pathlib import Path
import hjson
import os

from phd_visualizations.utils.units import unit_conversion
from phd_visualizations.utils import rename_signal_ids_to_var_ids

In [2]:
data_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/data')

filename_process_data = '20230508_solarMED.csv'
filename_process_data2 = '20230508_MED.csv'

paths = [data_path / filename_process_data, data_path / filename_process_data2]
sample_rate_key: str = '60s'



In [3]:
# Load varibles configuration

with open( Path("../data") / "variables_config.hjson") as f:
    vars_config = hjson.load(f)

In [4]:
def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()

    df.index = df.index.round('s')
    df = df.tz_localize('UTC')
    if 'time.1' in df.columns:
        df.drop(columns='time.1', inplace=True)

    # Select columns of 'object' data type
    object_columns = df.select_dtypes(include=['object'])

    # Print the column names
    logger.debug(object_columns.columns)  # %%

    if not object_columns.empty:
        logger.warning(f"Columns with object data type that will be deleted: {object_columns.columns}")
        
    # Delete columns with 'object' data type
    df = df.select_dtypes(exclude=['object'])
    
    # Drop duplicate index values from df
    duplicates_df = df.index.duplicated(keep='first').sum()
    logger.info(f"Number of duplicate index values in df: {duplicates_df}")
    df = df[~df.index.duplicated(keep='first')]

    return df

In [5]:
index_cols = ['time', 'TimeStamp']

if not isinstance(paths, list):
    paths = [paths]

for path in paths:
    if isinstance(path, str):
        path = Path(path)
    if not path.exists():
        raise FileNotFoundError(f"Path {path} does not exist.")

# Read reference dataframe
df = None
for index_col in index_cols:
    try:
        df = pd.read_csv(paths[0], parse_dates=True, index_col=index_col)
        break
    except Exception as e:
        pass
        # logger.error(f'Error while reading data from {paths[0]} with index_col={index_col}: {e}')

if df is None:
    raise RuntimeError(f'Failed to read data from CSV file with any of the provided index columns: {index_cols}')

df = process_dataframe(df)


2024-04-04 16:38:13.678 | DEBUG    | __main__:process_dataframe:14 - Index([], dtype='object')
2024-04-04 16:38:13.682 | INFO     | __main__:process_dataframe:24 - Number of duplicate index values in df: 0


In [6]:
# Read additional dataframes and concatenate them
for idx, path in enumerate(paths[1:]):
    df_aux = None
    for index_col in index_cols:
        try:
            df_aux = pd.read_csv(path, parse_dates=True, index_col=index_col)
            df_aux.index.names = ['time']
            break
        except Exception as e:
            logger.error(f'Error while reading data from {path} with index_col={index_col}: {e}')

    if df_aux is None:
        logger.error(f'Failed to read data from CSV file with any of the provided index columns for the {idx+2} path: {index_cols}')

    df_aux = process_dataframe(df_aux)

    # Find the common columns in both dataframes and drop them from the second
    common_columns = df.columns.intersection(df_aux.columns)
    df_aux = df_aux.drop(columns=common_columns)
    df = pd.concat([df, df_aux], axis=1)

# Preprocessing
# Sample every `sample_rate` seconds to reduce the size of the dataframe
df = df.resample(sample_rate_key).mean()

# Rename columns from signal_id to var_id
df = rename_signal_ids_to_var_ids(df, vars_config)

# Convert units to model units
df = unit_conversion(df, vars_config, input_unit_key='units_scada', output_unit_key='units_model')

# Filter out nans until first value in Tts
logger.warning(f"Removing {df['Tts_h_t'].isna().sum()} NaNs from the dataframe")
df = df[df['Tts_h_t'].notna()]

2024-04-04 16:38:13.838 | DEBUG    | __main__:process_dataframe:14 - Index([], dtype='object')
2024-04-04 16:38:13.841 | INFO     | __main__:process_dataframe:24 - Number of duplicate index values in df: 14
2024-04-04 16:38:14.004 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:55 - Signal id FT-SF-003 not found in dataframe columns.
2024-04-04 16:38:14.011 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tamb to C from C
2024-04-04 16:38:14.012 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_in to C from C
2024-04-04 16:38:14.012 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_in to C from C
2024-04-04 16:38:14.013 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_out to C from C
2024-04-04 16:38:14.014 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_out to C from C
2024-04-04 16:38:14.015 | DEBUG    | phd_visuali

# Compare columns in two dataframes

In [7]:

df1 = pd.read_csv(data_path / '20231030_solarMED.csv', parse_dates=True, index_col='TimeStamp')
df2 = pd.read_csv(data_path / '20230505_solarMED.csv', parse_dates=True, index_col='TimeStamp')

# Find columns that are in df1 but not in df2
columns_df1_not_df2 = df1.columns.difference(df2.columns)

logger.info(f"Columns in df1 but not in df2: {columns_df1_not_df2}")
logger.info(f"Columns 1: {df1.columns}")
logger.info(f"Columns 2: {df2.columns}")

ValueError: 'TimeStamp' is not in list

In [ ]:
df3 = pd.read_csv(Path('../data/calibration') / '20230707_20230710_datos_tanques.csv', parse_dates=True, index_col='TimeStamp')

logger.info(list(df3.columns))


# Nueva traza solarMED 
añadiendo datos necesarios para calibración de cada componente

In [ ]:
traza_original = [
    'FT-AQU-100', 'FT-AQU-101', 'TT-AQU-106', 'TT-AQU-107a', 'TT-AQU-109',
    'CT-DES-001', 'FT-DES-003', 'FT-DES-005', 'SW2TC1', 'TE-DES-015',
    'TT-DES-030', 'FT-SF-001', 'FT-SF-002', 'RE-SF-001', 'TT-SF-001',
    'TT-SF-002', 'TT-SF-003', 'TT-SF-004', 'TT-SF-005', 'TT-SF-006',
    'TT-SF-007', 'TT-SF-008', 'TT-SF-009', 'TT-SF-010'
]

additional_signals = [
    'TT-SF-018', 'TT-SF-019', 'FT-SF-005', 'TT-SF-024', 'TT-SF-025',
    'FT-SF-006', 'TT-SF-030', 'TT-SF-031', 'FT-SF-007', 'TT-SF-036',
    'TT-SF-037', 'FT-SF-008',  'FT-SF-002', 'TT-SF-009', 'TT-SF-010',
     'TT-DES-030', 'RE-SF-001', 'TT-SF-010', 'TT-SF-009', 'TT-SF-007',
    'TT-SF-008', 'TT-DES-030', 'FT-SF-002', 'FT-SF-001', 'TT-SF-010', 
    'TT-SF-009', 'TT-SF-007', 'TT-SF-008', 'TT-DES-030', 'FT-SF-002',
    'FT-SF-001', 'TT-SF-010',  'TT-SF-009', 'TT-SF-007', 'TT-SF-008',
    'TT-DES-030', 'FT-SF-002', 'FT-SF-001', 'UK-SF-P001-fq'
]

# Convert lists to sets for easier comparison
first_set = set(traza_original)
second_set = set(additional_signals)

# Find the elements in the second set that are not in the first set
difference = second_set - first_set

# Print the elements that are not present in the first list
for element in difference:
    print(element)